# Trabalho Prático 1 - Temas de Álgebra

### Mestrado em Matemática e Computação, Ano Letivo 2023-24

----------------------------------------

#### Enunciado

Enuncie e exemplifique o esquema de assinatura digital ECDSA.

https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm

Secção 3.3.10 de S Yan, "Number theory for Computing", Springer.

Se pretender usar o ECDSA aplicado no digest, sugerimos que use a construção da síntese dada pela função de hash do sagemath.

Por exemplo, a "hash" de uma mensagem pode ser obtida assim:
```sage
import hashlib
HH =hashlib.sha256(b"Nobody expects the Spanish Inquisition").hexdigest()
ZZ('0x'+HH)
```

Neste caso, usámos a função de hash SHA3 com 256 bits; independentemente do tamanho da mensagem original, a função de hash tem como imagem uma mensagem com no máximo 256 bits. 

---------------------------------------------------------

## * Algoritmo da Assinatura Digital de Curvas Elípticas (Elliptic Curve Digital Signature Algorithm - ECDSA) *

O _Elliptic Curve Digital Signature Algorithm (ECDSA)_ é um algoritmo de assinatura digital baseado em curvas elípticas, utilizado para garantir a segurança de dados em sistemas criptográficos.

Iremos definir os passos necessários para implementar o algoritmo ECDSA na ligunagem SAGE-Math, tendo partindo de uma curva elíptica exemplo e de uma mensagem exemplo que a Alice quer enviar para o Bob, onde o Bob terá de verificar se a assinatura gerada pela mensagem e enviada pela Alice se é válida.

### Passos do Algoritmo:

#### A) Criar a assinatura da mensagem (que se deseja enviar) sobre uma curva elíptica.

- 1) Definir uma curva elíptica de módulo _p_ (${F}_p$);
- 2) Obter um ponto gerador $G$ curva elíptica e a sua ordem $n$;
- 3) Gerar um número aleatório $privKey$ (_chave privada_) dentro do intervalo $[1, n-1]$\;
- 4) Calcular chave pública $pubKey$ = $privKey \cdot G$\;
- 5) Obter $z = HASH(mensagem)$;
- 6) Gerar um número aleatório $k$  dentro do intervalo $[1, n-1]$\;
- 7) Extrair as coordenadas do ponto $P$ = ($x_1$, $y_1$) = $k * G$;
- 8) Calcular $r \equiv x_1$ (mod $n$)\; 
    - (caso $r$=0, voltar ao passo 7); 
- 9) Calcular $s \equiv k^{-1} \cdot (z + r*$_privKey_$) \pmod{n}$\;
    - (caso $s$=0, voltar a dois passo 7);
- 10) Enviar a assinatura ($r$,$s$).

#### B) Verificar a validade da assinatura recebida.

- 1) Verificar se a _chave-pública_ ($pubKey$) não é igual ao elemento identidade ($O = n*G$). 
    - Se for igual, a assinatura é inválida.
    - __break__;
- 2) Verificar se a _chave-pública_ ($pubKey$) pertence à curva elíptica. 
    - Se não pertencer, a assinatura é inválida.
    - __break__;
- 3) Verificar se a $n * pubKey = O$ (elemento identidade).
    - Se não verificar a igualdade, a assinatura é inválida.
    - __break__;
- 4) Verificar se $r$ e $s$  pertencem ao intervalo $[1, n-1]$\;
    - Se não verificar a condição, a assinatura é inválida.
    - __break__;
- 5) Calcular $w \equiv s^{-1} \pmod{n}$\;
- 6) Obter $u_1 \equiv HASH(mensagem) \cdot w \pmod{n}$\;
- 7) Obter $u_2 \equiv r \cdot w \pmod{n}$\;
- 8) Calcular o ponto $(x_1, y_1) = u_1 \cdot G + u_2 \cdot pubKey$;
    - Se $(x_1, y_1) = O$ (elemento identidade), a assinatura é inválida;
    - __break__;
- 9) A assinatura é válida quando $r \equiv x_1 \pmod n$;

### __1)__ Definir uma curva elíptica exemplo para o problema, de módulo _p_ __primo__ ${F}_p$ com ordem __primo__.

Como queremos que este algoritmo seja capaz de codificar mensagens até __256 bits de segurança__ (basta obervar pelo uso da função de hash SHA3 com 256 bits), decidiu-se criar uma curva elíptica que fosse adequada para aplicação conjunta com o ECDSA e que respeite a condição de até 256 bits de segurança nas mensagens do sistema. Por isso, escolheu-se a curva elíptica __SECP256R1__ para aplicação neste trabalho.

Referência webgrafia da curva SECP256R1: https://www.herongyang.com/EC-Cryptography/Curve-secp256r1-for-256-Bit-ECC-Keys.html e https://neuromancer.sk/std/secg/secp256r1#

A curva elíptica __SECP256R1__ é altamente aplicada em Bitcoin, juntamente o algoritmo de critptografia ECDSA, tendo as seguintes características:

- de módulo primo __p__ = 115792089210356248762697446949407573530086143415290314195533631308867097853951 ($\mathbb{F}_n$);
- um ponto gerador da curva __G__ {x = 48439561293906451759052585252797914202762949526041747995844080717082404635286, y = 36134250956749795798585127919587881956611106672985015071877198253568414405109}.

Expressão geral da curva elíptica: $y^2 =x^3+ax+b \space mod \space p$.

Para os coeficentes usados por __SECP256R1__:

a = 115792089210356248762697446949407573530086143415290314195533631308867097853948

b = 41058363725152142129326129780047268409114441015993725554835256314039467401291

ficamos com uma curva elíptica de expressão: $y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 \space mod \space 115792089210356248762697446949407573530086143415290314195533631308867097853951$.


In [1]:
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951 # nº primo
Zp = IntegerModRing(p)

# coeficientes da curva elíptica
a = 115792089210356248762697446949407573530086143415290314195533631308867097853948
b = 41058363725152142129326129780047268409114441015993725554835256314039467401291

E = EllipticCurve(Zp, [a, b])
E

Elliptic Curve defined by y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 over Ring of integers modulo 115792089210356248762697446949407573530086143415290314195533631308867097853951

__2)__ Veriifcar a ordem, $n$, da curva elíptica, que tem de ser __primo__.

In [2]:
E.order()

115792089210356248762697446949407573529996955224135760342422259061068512044369

In [3]:
is_prime(E.order())

True

__3)__ Obter um ponto gerador da curva (G  pré-definida pela curva SECP256R1)

In [4]:
G = E(48439561293906451759052585252797914202762949526041747995844080717082404635286, 36134250956749795798585127919587881956611106672985015071877198253568414405109)
G

(48439561293906451759052585252797914202762949526041747995844080717082404635286 : 36134250956749795798585127919587881956611106672985015071877198253568414405109 : 1)

In [5]:
n = G.order() # ordem igual à da cruva elíptica definida
n

115792089210356248762697446949407573529996955224135760342422259061068512044369

In [6]:
n == E.order() # confirma-se que G é ponto gerador da curva elíptica

True

__4)__ Criar a chave privada aleatoriamente entre o intervalo $[1, n-1]$.

In [7]:
privKey = ZZ.random_element(1, n-1)
privKey

51618941118438689581961356997596064718315037041345235195354307221126195398097

In [8]:
# elemento identidade
O = n*G
O

(0 : 1 : 0)

__5)__ Criar a chave pública (ponto da curva), a partir do ponto gerador da curva G

In [9]:
pubKey = privKey * G
pubKey

(20656924604471837684249507890977932492835350587007348613201295763687925820039 : 47421052619301808015418340172657245955794505167063260664314998633513873917467 : 1)

__6)__ Vamos definir uma mensagem que Alice quer enviar para Bob

In [10]:
mens = b"Nobody expects the Spanish Inquisition"

__7)__ Obter a "hash" da mensagem, usando a função de hash SHA3 com 256 bits.

In [11]:
import hashlib

HH = hashlib.sha256(mens).hexdigest()
z = ZZ('0x'+HH)
z

36906822241210867014776754913837566155108892467980493944565876131413157803075

__8)__ Calcular o par ($r$, $s$) da assinatura digital, ou seja, cifrar a mensagem antes de enviar para o Bob

In [12]:
k = ZZ.random_element(1, n-1)
k

108921329171944491248263884777711651568849969702397774920775814571506768720541

In [13]:
# Ponto na curva
P = k * G
P

(103238776674795971235614296815023657170037018151015478129681992889445777856552 : 59264467711954751220598518857755403887122103230834435068369332609900327245686 : 1)

In [14]:
r=mod(P[0], n)
r

103238776674795971235614296815023657170037018151015478129681992889445777856552

Como o r != 0, podemos avançar para o próximo cálculo:

In [15]:
s = mod(inverse_mod(k, n)*(z + privKey * r), n)
s

56032535769656753484977299212059895677396409968579393909954086631752611922608

Também $s$ != 0, pelo que obtivemos a assinatura digital ($r$, $s$) será enviada para o Bob:

In [16]:
(r, s)

(103238776674795971235614296815023657170037018151015478129681992889445777856552,
 56032535769656753484977299212059895677396409968579393909954086631752611922608)

__9)__ Bob tem de verificar se a assinatura recebida é válida.

Bob precisa de ter acesso à chave-pública, curva elíptica e à mensagem original para verificar a assinatura da Alice.

In [17]:
not(pubKey is O) # coordenadas de pubKey válidas

True

In [18]:
# verificar se a chave pública pertence à curva elíptica
x = pubKey[0]
y = pubKey[1]
(y**2) % p == (x**3 + a*x + b) % p

True

In [19]:
n*pubKey == O # verificar se n*Q é igual ao elemento identificador

True

In [20]:
1 <= r <= n-1 and 1 <= s <= n-1

True

Caso algumas das condições anteriores devolvê-se o valor "_False_", Bob podia concluir imediatamente que a assinatura que recebeu da Alice era inválida.

In [21]:
HH_Bob = hashlib.sha256(mens).hexdigest() # usa-se a mesma função de hash que a Alice aplicou
z_Bob = ZZ('0x'+HH_Bob)
z_Bob

36906822241210867014776754913837566155108892467980493944565876131413157803075

In [22]:
from gmpy2 import mpz

w = inverse_mod(mpz(s), int(n)) # inversa de s, módulo n ()
w

72588597651539317021072227046078153031801525523449709874758641605924301767515

In [23]:
u1 = Integer((z_Bob * w) % n)
u1

31437790731975584752402079354272134937548074102489108292790209416849601709548

In [24]:
u2 = Integer((r * w)% n)
u2

114450416645227272688866324319260304988059188058634936289505498427768287368325

In [25]:
# ponto da curva
ponto = (u1*G) + (u2*pubKey)
ponto

(103238776674795971235614296815023657170037018151015478129681992889445777856552 : 59264467711954751220598518857755403887122103230834435068369332609900327245686 : 1)

In [26]:
# vamos verificar se o ponto da curva é igual ao elemento identidade da curva elíptica
not(ponto == O)

True

In [27]:
x1 = ponto[0]
mod(x1, n) == r

True

Com o resultado anterior, Bob pode confirmar que a assinatura recebida é __VÁLIDA__!

Deste modo, mostramos um exemplo da implementação de ECDSA que nos permite observar as etapas necessárias para tal.